In [1]:
import os
import re
import pandas as pd
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

In [2]:
ETF = 'NDX'
ETF_3x = 'TQQQ'

In [3]:
final_result = pd.read_excel(
    '{}_leverage_simulation.xlsx'.format(ETF),
    )

In [4]:
final_result.head(3)

,Date,NDX,simulated_TQQQ,TQQQ
0,1985-10-02,110.824997,0.299116,NaN
1,1985-10-03,110.870003,0.299175,NaN
2,1985-10-04,110.074997,0.298136,NaN


# the return

In [5]:
daily_invest_amount = 1000

invest = pysqldf(u"""
    select 
    Date,
    {} AS price,
    case
        when {} is not null then {}
        else simulated_{}
    end as price_3x
    from final_result
    """.format(
        ETF,
        ETF_3x,
        ETF_3x,
        ETF_3x,
    ))

invest1 = pysqldf(u"""
    select *,
    {} as daily_invest_amount,
    {}/price as shares,
    {}/price_3x as shares_3x    
    from invest
    order by Date asc
    """.format(
    daily_invest_amount,
    daily_invest_amount,
    daily_invest_amount))

In [6]:
invest1.head(3)

,Date,price,price_3x,daily_invest_amount,shares,shares_3x
0,1985-10-02,110.824997,0.299116,1000,9.023235,3343.180430
1,1985-10-03,110.870003,0.299175,1000,9.019572,3342.523902
2,1985-10-04,110.074997,0.298136,1000,9.084715,3354.168665


In [7]:
days_duration = int(20*250)

final_result['id'] = final_result.index

first_last_days = pysqldf(u"""
    select 
    f.Date as invest_date_first, 
    date_2500_days_later.Date as invest_date_last
    from final_result as f
    join final_result as date_2500_days_later on date_2500_days_later.id = f.id+{}
    """.format(days_duration)).to_dict('records')

print(days_duration)

5000


In [ ]:
returns = []

for r in first_last_days:
    invest_date_first = r['invest_date_first']
    invest_date_last = r['invest_date_last']
    
    print('processing data between {} and {}'.format(
    invest_date_first,
    invest_date_last,
    ))

    #invest_date_first = '2013-01-01'
    #invest_date_last = '2013-01-05'

    window = pysqldf(u"""
        select *
        from invest1
        where Date >= '{}'
        and Date <= '{}'
        """.format(
        invest_date_first, 
        invest_date_last
        ))

    total_df = pysqldf(u"""
        select 
        sum(daily_invest_amount) as invest_amount,
        sum(shares) as shares,
        sum(shares_3x) as shares_3x
        from window
        """)

    invest_amount = total_df['invest_amount'][0]
    shares = total_df['shares'][0]
    shares_3x = total_df['shares_3x'][0]

    ###

    first_date = pysqldf(u"""
        select *  
        from window
        order by Date asc
        limit 1
        """)

    first_price = first_date['price'][0]
    first_price_3x = first_date['price_3x'][0]

    last_date = pysqldf(u"""
        select *  
        from window
        order by Date desc
        limit 1
        """)

    last_price = last_date['price'][0]
    last_price_3x = last_date['price_3x'][0]

    ###

    all_in_return = last_price/first_price-1
    all_in_3x_return = last_price_3x/first_price_3x-1

    cost_averaging_return = shares*last_price/invest_amount-1
    cost_averaging_3x_return = shares_3x*last_price_3x/invest_amount-1

    record = {
        'invest_date_first':invest_date_first,
        'invest_date_last':invest_date_last,
        'all_in_return':all_in_return,
        'all_in_3x_return':all_in_3x_return,
        'cost_averaging_return':cost_averaging_return,
        'cost_averaging_3x_return':cost_averaging_3x_return,    
    }
    
    returns.append(record)


###    

returns_duration_df = pd.DataFrame(returns)

returns_duration_df.to_excel(
    '{}_returns_{}d.xlsx'.format(
        ETF,
        days_duration),
    index = False
    )

processing data between 1985-10-02 and 2005-07-27
processing data between 1985-10-03 and 2005-07-28
processing data between 1985-10-04 and 2005-07-29
processing data between 1985-10-07 and 2005-08-01
processing data between 1985-10-08 and 2005-08-02
processing data between 1985-10-09 and 2005-08-03
processing data between 1985-10-10 and 2005-08-04
processing data between 1985-10-11 and 2005-08-05
processing data between 1985-10-14 and 2005-08-08
processing data between 1985-10-15 and 2005-08-09
processing data between 1985-10-16 and 2005-08-10
processing data between 1985-10-17 and 2005-08-11
processing data between 1985-10-18 and 2005-08-12
processing data between 1985-10-21 and 2005-08-15
processing data between 1985-10-22 and 2005-08-16
processing data between 1985-10-23 and 2005-08-17
processing data between 1985-10-24 and 2005-08-18
processing data between 1985-10-25 and 2005-08-19
processing data between 1985-10-28 and 2005-08-22
processing data between 1985-10-29 and 2005-08-23


def calculate_return(
    invest_date_first,
    invest_date_last,
    ):

pysqldf(u"""
    select *  
    from invest
    where Date in ('2001-04-24', '2009-12-15')
    """)

return_df.plot(
    x ='Date', 
    y=[
        'accumulated_return', 
        'accumulated_return_3x', 
    ], grid = True)

return_df.plot(
    x ='Date', 
    y=[
        'price', 
        'price_3x', 
    ], grid = True)

END